In [ ]:
import camelot
import pandas as pd
import os



In [ ]:

# drive.mount('/content/drive')

# Set the directory containing the PDF files
pdf_dir = '/Users/vamshi/Documents/GenAI/camelot2/content/pdf_files'

# Create the output directory for CSV files
csv_dir = '/Users/vamshi/Documents/GenAI/camelot2/content/csv_files'
os.makedirs(csv_dir, exist_ok=True)

# Process each PDF file in the directory
pdf_files = [f for f in os.listdir(pdf_dir) if f.endswith('.pdf')]




In [ ]:
import os

# Iterate through each PDF file
for pdf_file in pdf_files:
    print(f"Processing PDF file: {pdf_file}")
    pdf_path = os.path.join(pdf_dir, pdf_file)
    tables = camelot.read_pdf(pdf_path, pages='all')
    
    # Create a DataFrame for the current PDF file
    pdf_tables_df = pd.concat([table.df for table in tables])
    
    # Generate the CSV file name based on the PDF file name
    csv_file_name = os.path.splitext(pdf_file)[0] + ".csv"
    csv_file_path = os.path.join(csv_dir, csv_file_name)
    
    # Export the DataFrame to a CSV file
    pdf_tables_df.to_csv(csv_file_path, index=False)
    print(f"Exported tables from {pdf_file} to {csv_file_path}")


In [ ]:
# Step 2: Load CSV and Process with Langchain
from langchain_community.document_loaders import DirectoryLoader
from langchain.chains import RetrievalQA
from langchain_community.llms import Ollama
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import Chroma



In [ ]:
# Load documents from the csv_dir directory
loader = DirectoryLoader(csv_dir)
data = loader.load()



In [ ]:
# Split the text into manageable chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

# Generate embeddings for the documents
oembed = OllamaEmbeddings(base_url="http://localhost:11434", model="nomic-embed-text")
vectorstore = Chroma.from_documents(documents=all_splits, embedding=oembed)



In [ ]:
# Set up the LLM and QA chain
llm = Ollama(base_url="http://localhost:11434/", model="llama3")
qachain = RetrievalQA.from_chain_type(llm, retriever=vectorstore.as_retriever())



In [ ]:
# Query the data
question = """ Net Revenue  of ambuja cement """
response = qachain.invoke({"query": question})
print(response)